In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep # control the crawl rate to avoid hammering the servers with too many requests
from random import randint
import re # regular expression

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities # dealing with pop-up permissions
from selenium.webdriver.firefox.options import Options

In [2]:
# what i need to get from the reviews
# 1) stars (0 - 5) ## "data-comp="StarRating" <div aria-label="4 stars">
# 2) reviewer id ## <a href="users/Tarah84" ...> under <strong data-at="nickname">____</strong>
# 3) reviewer features ## two lines under "nickname"
# 4) purchased shade ## line after headline of the review ("<h3 class>")

reviewer_id = []
rating = []
reviewer_feature = []
purchased_shade = []

cols = {
    "reviewer_id": reviewer_id, "rating": rating, 
    "reviewer_feature": reviewer_feature, "purchased_shade": purchased_shade
    }

fenty_dat = pd.DataFrame(cols)


In [3]:
geoBlocked = webdriver.FirefoxOptions()
geoBlocked.set_preference("geo.prompt.testing", True)
geoBlocked.set_preference("geo.prompt.testing.allow", False)
driver = webdriver.Firefox(options=geoBlocked)
driver.get("https://www.sephora.com/product/pro-filtr-soft-matte-longwear-foundation-P87985432?skuId=2164697")
sleep(3)
driver.execute_script("window.scrollTo(0, 3000)") 
sleep(2)
driver.execute_script("window.scrollTo(0, 4500)") 

In [4]:
# preparing to click
# driver.find_elements_by_css_selector('div[data-comp="Pagination Flex StyledComponent BaseComponent "')

In [5]:
list_of_reviews = driver.find_elements_by_css_selector('div[data-comp="Review StyledComponent BaseComponent "]')

list_of_reviews[0].find_elements_by_css_selector('strong[data-at="nickname"]')[0].text


def append_new_data(data = fenty_dat, list_of_reviews = list_of_reviews):
    shade = ""
    for i in list_of_reviews:
        if i.find_elements_by_css_selector('img[src*="https://www.sephora.com/productimages/"]+span') != []:
            shade = i.find_elements_by_css_selector('img[src*="https://www.sephora.com/productimages/"]+span')[0].text
        else:
            shade = None

        new_col = {
            "reviewer_id": i.find_elements_by_css_selector('strong[data-at="nickname"]')[0].text,
            "rating": int(i.find_elements_by_css_selector('div[data-comp="StarRating "]')[0].get_attribute("aria-label")[0]),
            "reviewer_feature": i.find_elements_by_css_selector('strong[data-at="nickname"]+span')[0].text,
            "purchased_shade": shade
        }
        # print(new_col)
        data = data.append(new_col,ignore_index=True)
    return data

/var/folders/b8/qhvvp5cj1hj_nj6sgf68yv3r0000gn/T/ipykernel_61519/2311291850.py:1: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  list_of_reviews = driver.find_elements_by_css_selector('div[data-comp="Review StyledComponent BaseComponent "]')
/opt/homebrew/Caskroom/miniforge/base/envs/foundation/lib/python3.9/site-packages/selenium/webdriver/remote/webelement.py:502: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")


In [30]:

driver.find_element_by_css_selector('ul[data-comp="Pagination Flex StyledComponent BaseComponent "]:nth-child(8)').text

/var/folders/b8/qhvvp5cj1hj_nj6sgf68yv3r0000gn/T/ipykernel_61519/1726895540.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector('ul[data-comp="Pagination Flex StyledComponent BaseComponent "]:nth-child(8)[button]').text


NoSuchElementException: Message: Unable to locate element: ul[data-comp="Pagination Flex StyledComponent BaseComponent "]:nth-child(8)[button]
Stacktrace:
WebDriverError@chrome://remote/content/shared/webdriver/Errors.jsm:181:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.jsm:393:5
element.find/</<@chrome://remote/content/marionette/element.js:299:16


In [6]:
fenty_dat = append_new_data()

In [7]:
fenty_dat

,reviewer_id,rating,reviewer_feature,purchased_shade
0,mbrid,3.0,"Hazel eyes, Brunette hair, Medium skin tone, C...",270 - for medium to tan skin with cool undertones
1,tidizzle,5.0,"Green eyes, Blonde hair, Light skin tone, Norm...",185 - for light to medium skin with neutral un...
2,bisousvanille,5.0,"Brown eyes, Black hair, Olive skin tone, Combi...",345 - for tan skin with warm olive undertones
3,Jenninipls,1.0,"Hazel eyes, Brunette hair, Porcelain skin tone...",100 - for very fair skin with neutral undertones
4,madkaita,5.0,"Blue eyes, Blonde hair, Fair skin tone, Combin...",180 - for light skin with warm undertones
5,aylincp,5.0,"Hazel eyes, Brunette hair, Light skin tone, Co...",None
